![header](https://i.imgur.com/I4ake6d.jpg)

# IN SITU BALTIC SEA TRAINING

<div style="text-align: right"><i> 13-05-Part-five-out-of-five </i></div>

# BAL `NRT` product/dataset: managing files (thermosal)

<h1>Table of Contents<span class="tocSkip"></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction">Introduction</a></span></li>
        <li>
            <span><a href="#Setup" data-toc-modified-id="Setup">Setup</a></span>
            <ul>
                <li><span><a href="#Python-packages" data-toc-modified-id="Python-packages">Python packages</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Thermosal-(TS)-data" data-toc-modified-id="Thermosal-(TS)-data">Thermosal (TS) data</a></span>
            <ul>
                <li><span><a href="#Trajectory-animation" data-toc-modified-id="Trajectory-animation">Trajectory-animation</a></span></li>
                <li><span><a href="#Along-track-variable-evolution" data-toc-modified-id="Along-track-variable-evolution">Along track variable evolution</a></span></li>
        <li><span><a href="#Overall-variable-evolution" data-toc-modified-id="Overall-variable-evolution">Overall variable evolution</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Wrap-up" data-toc-modified-id="Wrap-up">Wrap-up</a></span></li>
        <li><span><a href="#Feedback-survey" data-toc-modified-id="Feedback-survey">Feedback survey</a></span></li>
    </ul>
</div>

## Introduction

According to the [13-01-NearRealTtime-product-collections-overview.ipynb](13-01-NearRealTtime-product-collections-overview.ipynb) one of the data source types available are the Thermosal. Please use the notebook [13-02-NearRealTtime-product-subsetting-download](13-02-NearRealTtime-product-subsetting-download.ipynb) to download some files from Thermosal ('TS' data type) and let's check its data. <br> If you wanna skip the downloading part you can use the netCDF files available <a href="data" target="_blank">here</a> instead.    

## Setup

### Python packages

For the notebook to properly run we need to first load the next packages available from the Jupyter Notebook Ecosystem. Please run the `next cell`:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import IPython
import datetime
import numpy as np
import xarray
import folium
from folium import plugins
from dateutil.relativedelta import relativedelta
import branca
%matplotlib inline

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If any of them raises any error it means you need to install the module first. For doing so please:
1. Open a new cell int he notebook
2. Run <i>`!conda install packageName --yes`</i> or <i>`!conda install -c conda-forge packageName --yes`</i> or <i>`!pip install packageName`</i>
3. Import again!
<br><br>
Example: <i>how-to-solve import error for json2html module </i>

![region.png](img/errorImporting.gif)

### Auxiliary functions

Please `run the next cells` to load into memory some useful functions we will use later on:

In [ ]:
def get_subset(start,end,ds):
    #Subsets a dataset (ds) from start to end dates
    i_start = ds['TIME'].astype(str).values.tolist().index(ds['TIME'][ds['TIME'].astype(str).str.contains(start) == True].astype(str).values[0])
    i_end = ds['TIME'].astype(str).values.tolist().index(ds['TIME'][ds['TIME'].astype(str).str.contains(end) == True].astype(str).values[-1])
    return ds.isel(TIME=slice(i_start, i_end),LATITUDE=slice(i_start, i_end),LONGITUDE=slice(i_start, i_end),POSITION=slice(i_start, i_end))

## Thermosal (TS) data

Thermosalor Thermosalinographs are devices on board of vessels that measure watter intakes (i.e temperature, conductivity etc) as it moves. The resulting data feature is a trajectory-like time serie.

Let's see the data of one of the available thermosal in the BS.<br>`Run the next cell` to see the thermosal files already available in the /data folder:

In [ ]:
IPython.display.IFrame('data/files/TS', width='100%', height=350)

### Reading the file

`Set one` of the above available `file name` and `run the next cells`:

In [ ]:
file = 'GL_TS_TS_FNHO.nc'
path = os.path.join(os.path.join(os.getcwd(), 'data', 'files','TS'), file)

In [ ]:
ds = xarray.open_dataset(path)
ds.close()
ds

The above one is an overview of the content of the file: variables, dimensions, coordinates, global attributes...
<br>Let's list now the available variables: `run the next cell`

In [ ]:
for var in ds.variables:
    print(var + ':' + ds[var].attrs['long_name'])

Let's watch out the temporal coverage: `run the next cell`

In [ ]:
start = datetime.datetime.strptime(ds.attrs['time_coverage_start'], '%Y-%m-%dT%H:%M:%SZ')
end = datetime.datetime.strptime(ds.attrs['time_coverage_end'], '%Y-%m-%dT%H:%M:%SZ')
years = relativedelta(end, start).years
print(str(years)+' years ('+ds.attrs['time_coverage_start']+'/'+ds.attrs['time_coverage_end']+')')

The number of years is too large so we will just continue working with a sample: ony one year. 
<br> Choose the year in the next and `run the next cell`:

In [ ]:
targeted_year = '2019'
yearSubset = get_subset(targeted_year,targeted_year, ds)

### Data visualization

#### Trajectory animation

As stated before, the thermosal is on board of a mobile platform (a vessel).
<br>Let's create now a geojson feature representing the vessel; we will populate it next: `run the next cell`

In [ ]:
vessel = {
    'type': 'Feature',
    'geometry': {
        'type': 'LineString',
        'coordinates': []
    },
    'properties': {
        'times': [],
    }
}

Before passing it all original coordinates, let's check if all of them are flagged as good or not!

In In Situ TAC netCDFs all variables are linked to another called the same plus '_QC'. This 'twin' variable contains a quality flag for each value in the paired variable.`run the next cell` to check the flag values convention:

In [ ]:
pd.DataFrame(data=yearSubset['POSITION_QC'].attrs['flag_values'],
             index=yearSubset['POSITION_QC'].attrs['flag_meanings'].split(' '), columns=['quality flag'])

Users are recommended to use only the data flagged as 1, they so called 'good data'. Let's then check the available flags for the coordinates (time and position) to see if we need to get rid of not-good values: `run the next cells`

In [ ]:
yearSubset['POSITION_QC'].plot(aspect=2, size=5)

In [ ]:
set(subset['POSITION_QC'].values.tolist())

From above, we see that there are some QC flags values different from 1, meaning we have to filter out the not-good values before anything else!: `run the next cell`

In [ ]:
lats = yearSubset['LATITUDE'].where(yearSubset['POSITION_QC'] == 1).values.tolist()
lons = yearSubset['LONGITUDE'].where(yearSubset['POSITION_QC'] == 1).values.tolist()
times = yearSubset['TIME'].astype(str).values.tolist()

Let's populate the geojson:

In [ ]:
for time, lat, lon in zip(times, list(zip(*lats))[0], list(zip(*lons))[0]):
    base = [lat,lon]
    if(any(x is None for x in base)):
        continue
    if(any(np.isnan(x) for x in base)):
        continue
    vessel['properties']['times'].append(time[:19])
    vessel['geometry']['coordinates'].append([lon, lat])

In [ ]:
mean_lat, mean_lon = np.nanmean(lats), np.nanmean(lons)
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=3)
marker = plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': [vessel],
}, add_last_point=True, loop=False, period='PT1H').add_to(m)
m

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If you do not see any map when running the next cell please change your navigator (try chrome!).

### Along track variable evolution

Water properties varies a lot from Baltic Sea to the Mediterranean Sea. So let's focus in one of these transects next:

In [ ]:
start,end = '2019-07-01', '2019-07-10'
BAL2MEDSubset = get_subset(start,end,yearSubset)

Let's focus on one of the variables to visualize its data!: `set one and run the next cell`

In [ ]:
param = 'PSAL'

In [ ]:
BAL2MEDSubset[param+'_QC'][:,0].plot()

In [ ]:
set(BAL2MEDSubset[param+'_QC'][:,0].values.tolist())

Let's get only the good data for the only depth (0):

In [ ]:
var = BAL2MEDSubset[param][:,0].where(BAL2MEDSubset[param+'_QC'][:,0] == 1).values.tolist()

Let's set a colormap:

In [ ]:
linear_cmap = branca.colormap.LinearColormap(['green', 'yellow', 'red'],vmin=np.nanmin(var), vmax=np.nanmax(var))
linear_cmap

Let's plot the temperature values along the trajectory:

In [ ]:
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=5)
lats = BAL2MEDSubset['LATITUDE'].values.tolist()
lons = BAL2MEDSubset['LONGITUDE'].values.tolist()
for k in range(0,len(times)-1):
    try:
        color = linear_cmap(var[k])
        folium.CircleMarker([lats[k], lons[k]], radius=2,color=color).add_to(m)
    except Exception as e:
        pass
m.fit_bounds(m.get_bounds())
colormap = branca.colormap.LinearColormap(['green', 'yellow', 'red']).scale(int(np.nanmin(var)), int(np.nanmax(var))).to_step(6)
colormap.caption = 'Temperature variation along the vessel track'
m.add_child(colormap)
m

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If you do not see any map when running the next cell please change your navigator (try chrome!).

### Overall variable evolution

In [ ]:
BAL2MEDSubset[param][:,0].where(subset[param+'_QC'][:,0] == 1).plot(aspect=3, size=5, marker='o', color='k')

***

## Wrap-up

So far you should already know how to deal with trajectory-like Time Serie data from drifting buoys. <br> `If you don't please ask us! it is the moment!`

---



## Feedback survey

<div class="alert alert-block alert-success">
    <b>CONGRATULATIONS</b><br>

***
**IF IT'S 202025 PAST MIDDAY, PLEASE READ CAREFULLY BELOW LINES (ACTION FROM YOUR SIDE)**
***    
This training course is over but we'd love to hear from you about how we could improve it (topics, tools, storytelling, format, speed etc). 

We have prepared a little questionnaire to gather all your inputs, available here (just click on the hyperlink or execute the very last cell and click on `Answer`):
- https://tiny.cc/training-bal-insitu

We do thank you in advance for your kind collaboration :)

Greetings <3

In [ ]:
IPython.display.IFrame('https://tiny.cc/tw-bal-insitu', width=900, height=500)